ref
1. Kaggle decision tree
https://www.kaggle.com/code/marklvl/decision-tree-regressor-on-bike-sharing-dataset
2. Avoid $R^2$ to evaluate regression model
https://towardsdatascience.com/avoid-r-squared-to-judge-regression-model-performance-5c2bc53c8e2e

syntax ref
1. https://stackoverflow.com/questions/33643843/cant-drop-nan-with-dropna-in-pandas

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [82]:
import os
import glob
import pandas as pd
import numpy as np


from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score, cross_val_predict
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor as rfr


import matplotlib.pyplot as plt
import seaborn as sns
sns.set()

In [6]:
FOLDERNAME_LEV3 = "/content/drive/MyDrive/서울대/2022-1/기계시스템설계1/data/LEV3"


# Utility functions

In [7]:
#utility functions

def load_data(FOLDRENAME):
    directory = os.fsdecode(FOLDRENAME)
    files = []

    for file in os.listdir(directory):
        filename = os.fsdecode(file)
        if filename.endswith(".csv"):
            file_abs_path = os.path.join(FOLDRENAME, file)
            files.append(file_abs_path)
    
    return pd.concat(map(pd.read_csv, files), ignore_index=True)

# Loading Data

In [8]:
df = load_data(FOLDERNAME_LEV3)

In [9]:
df.head()

,IDX0002SantafeTMOOO19_KMHS281BBKU151711_LSMITI137M000D01995FF,CAL_Time_sec,CAL_SerialTime_none,AUX_RTCDate_YYYYMMDD,AUX_DeviceDate_YYMMDD,AUX_DeviceDate_hhmmss,OBD_CalEngLoad_perc,OBD_EngCoolantTemp_degC,OBD_InManiAbsPress_kPa,OBD_EngineSpeed_rpm,...,AUX_MagneticFieldZ_uT,AUX_sensSCVPosi_V,AUX_sensEGRValvePosi_V,CAL_ExhFlowrate_gps,CAL_CO2Flowrate_gphr,CAL_NOxFlowrateLNTInletValid_gps,CAL_NOxFlowrateLNTOutletValid_gps,CAL_NOxFlowrateSCROutletValid_gps,CAL_sensBaroPressSpdCorr_mbar,CAL_AltFromBarometer_m
0,1,0,737902.412280,2020-04-21 09:53:41.000,20200421,95340.99999,NaN,NaN,NaN,NaN,...,7.296222,3.930081,0.260084,NaN,0.0,0.0,0.0,0.0,1003.749634,0.000000
1,2,1,737902.412292,2020-04-21 09:53:42.000,20200421,95342.00000,0.0,8.0,99.0,0.0,...,6.651423,3.931051,0.261396,0.0,0.0,0.0,0.0,0.0,1003.756198,-0.046019
2,3,2,737902.412303,2020-04-21 09:53:43.000,20200421,95343.00000,0.0,8.0,99.0,0.0,...,7.200000,3.928769,0.261396,0.0,0.0,0.0,0.0,0.0,1003.758468,-0.062014
3,4,3,737902.412315,2020-04-21 09:53:44.000,20200421,95344.00000,0.0,8.0,99.0,0.0,...,7.150495,3.926828,0.262994,0.0,0.0,0.0,0.0,0.0,1003.756747,-0.050109
4,5,4,737902.412326,2020-04-21 09:53:45.000,20200421,95345.00000,0.0,8.0,99.0,0.0,...,6.850735,3.932021,0.261054,0.0,0.0,0.0,0.0,0.0,1003.751961,-0.016767


In [10]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 24595 entries, 0 to 24594
Columns: 132 entries, IDX0002SantafeTMOOO19_KMHS281BBKU151711_LSMITI137M000D01995FF to CAL_AltFromBarometer_m
dtypes: float64(124), int64(6), object(2)
memory usage: 24.8+ MB


In [11]:
df.describe()

,IDX0002SantafeTMOOO19_KMHS281BBKU151711_LSMITI137M000D01995FF,CAL_Time_sec,CAL_SerialTime_none,AUX_DeviceDate_YYMMDD,AUX_DeviceDate_hhmmss,OBD_CalEngLoad_perc,OBD_EngCoolantTemp_degC,OBD_InManiAbsPress_kPa,OBD_EngineSpeed_rpm,OBD_VehicleSpeed_kmph,...,AUX_MagneticFieldZ_uT,AUX_sensSCVPosi_V,AUX_sensEGRValvePosi_V,CAL_ExhFlowrate_gps,CAL_CO2Flowrate_gphr,CAL_NOxFlowrateLNTInletValid_gps,CAL_NOxFlowrateLNTOutletValid_gps,CAL_NOxFlowrateSCROutletValid_gps,CAL_sensBaroPressSpdCorr_mbar,CAL_AltFromBarometer_m
count,24595.000000,24595.000000,24595.000000,2.459500e+04,24595.000000,24515.000000,24515.000000,24515.000000,24515.000000,24515.000000,...,24595.000000,24595.000000,24595.000000,24516.000000,24595.000000,24595.000000,24595.000000,24595.000000,24595.000000,24595.000000
mean,3076.056516,3075.056516,737903.954084,2.020042e+07,103798.164749,45.664127,79.060902,114.334194,1317.166908,49.398653,...,1.765689,3.382409,2.690374,20.113594,7966.967142,0.011319,0.006664,0.000812,1004.539622,4.688337
std,1777.098480,1777.098480,1.118941,1.124449e+00,5467.682993,26.907614,16.976225,24.594252,364.340053,32.273799,...,9.088363,0.459987,0.926204,13.161787,8609.559403,0.022665,0.017420,0.003676,4.047683,26.970959
min,1.000000,0.000000,737902.412280,2.020042e+07,92810.999991,0.000000,7.000000,56.000000,0.000000,0.000000,...,-106.169141,1.528638,0.255990,0.000000,0.000000,0.000000,0.000000,0.000000,985.031832,-35.255621
25%,1538.000000,1537.000000,737903.407749,2.020042e+07,100903.000000,27.058824,83.000000,101.000000,1211.896860,25.189344,...,2.638573,3.148333,2.445085,11.734617,1907.104591,0.000989,0.000053,0.000000,1002.843570,-13.348989
50%,3075.000000,3074.000000,737904.407512,2.020042e+07,103440.000000,49.411765,85.000000,109.000000,1376.389650,52.000000,...,3.702950,3.364418,3.122730,18.309924,5409.601116,0.002251,0.000621,0.000006,1005.387653,-3.165688
75%,4612.000000,4611.000000,737905.396916,2.020042e+07,110017.000000,67.896083,86.000000,120.603942,1526.546235,71.000000,...,4.515304,3.860122,3.253924,25.813431,11951.359177,0.010323,0.004574,0.000253,1007.368964,16.986087
max,6352.000000,6351.000000,737905.468079,2.020042e+07,113544.000000,99.929661,90.000000,255.000000,3201.986664,129.000000,...,57.300000,3.948102,4.115916,149.687361,89600.459149,0.278177,0.384286,0.106603,1011.316936,135.750057


# Preprocessing

In [111]:
#set input, output features

input_features = [
    'AUX_NOxLNInlet_ppm', 
#     'AUX_NOxLNTOutlet_ppm',
#    'CAL_Time_sec', 
#    'AUX_RTCDate_YYYYMMDD', 
   'OBD_CalEngLoad_perc',
    'OBD_EngCoolantTemp_degC', 
    'OBD_EngineSpeed_rpm', 
#    'OBD_VehicleSpeed_kmph', 
#    'OBD_ActEngPerTorque_perc',
#    'OBD_EngRefTorque_Nm', 
#    'OBD_MAFSensor_gps', 
#    'OBD_EGT11_degC', 
#    'OBD_EGT12_degC',
#    'OBD_DPFDiffPress_kPa', 
#    'AUX_NOxSCROutlet_ppm', 
#    'AUX_lambdaLNTInlet_none', 
#    'AUX_lambdaLNTOutlet_none',
#    'AUX_lambdaSCROutlet_none', 
    'AUX_O2FracLNTInlet_volPerc', 
    'AUX_O2FracLNTOutlet_volPerc',
#    'AUX_O2FracSCROutlet_volPerc',
#    'AUX_SensAmbTemp_degC',
#    'AUX_SensAmbRH_perc',
#    'AUX_SensBaroPress_kPa',
    'AUX_SensTempTurbinOutlet_degC',
#    'AUX_SensTempInManiInlet_degC',
#    'AUX_SensTempTurbinInlet_degC',
    'AUX_SensTempLNTOutlet_degC',
#    'AUX_SensTempSCROutlet_degC',
#    'AUX_SensTempCompressorOutlet_degC',
#    'AUX_SensPressEGRCoolerInlet_absBar',
#    'AUX_SensDiffPressDPF_kPa',
#    'AUX_SensDiffPressSCR_kPa',
    'CAL_ExhFlowrate_gps',
#    'CAL_CO2Flowrate_gphr',
   'CAL_NOxFlowrateLNTInletValid_gps',
#    'CAL_NOxFlowrateLNTOutletValid_gps',
#    'CAL_NOxFlowrateSCROutletValid_gps'
]

output_features = [
    'CAL_NOxFlowrateLNTOutletValid_gps',
    'AUX_O2FracLNTOutlet_volPerc',
    'AUX_SensTempLNTOutlet_degC',
]

In [112]:
# dorp null value
df.dropna(inplace = True)

#cleaning data
RPM_min = 100                      # excluding engine-stop
NOx_max = 1649                     # excluding clipped NOx data (since the sensor maxed out at 1650 ppm)

df = df.loc[((df['OBD_EngineSpeed_rpm'] >= RPM_min) & (df['AUX_NOxLNInlet_ppm'] <= NOx_max))]

In [113]:
X = df[input_features]
y = df[output_features]

In [114]:
X.describe()

,AUX_NOxLNInlet_ppm,OBD_CalEngLoad_perc,OBD_EngCoolantTemp_degC,OBD_EngineSpeed_rpm,AUX_O2FracLNTInlet_volPerc,AUX_O2FracLNTOutlet_volPerc,AUX_SensTempTurbinOutlet_degC,AUX_SensTempLNTOutlet_degC,CAL_ExhFlowrate_gps,CAL_NOxFlowrateLNTInletValid_gps
count,23833.000000,23833.000000,23833.000000,23833.000000,23833.000000,23833.000000,23833.000000,23833.000000,23833.000000,23833.000000
mean,241.385083,46.364280,80.316065,1339.192085,9.910926,9.941003,264.011010,246.636286,20.329752,0.011278
std,303.479140,26.349974,14.127071,320.095356,5.541563,5.497070,97.849650,112.170658,12.834047,0.022184
min,0.000000,0.000000,7.000000,730.740352,-5.135187,-5.954211,10.937500,7.524541,4.938342,0.000000
25%,64.400000,27.611758,83.000000,1230.723071,5.838882,5.838882,190.125000,186.451848,11.968339,0.001034
50%,112.044811,50.006462,85.000000,1378.145994,8.866294,8.846786,243.911923,215.573093,18.452233,0.002360
75%,286.300000,67.989687,86.000000,1531.652762,12.747460,12.875421,319.312500,285.454121,25.799537,0.010628
max,1648.988204,99.929661,90.000000,3201.986664,22.402287,21.366533,616.816691,645.811458,149.687361,0.278177


In [115]:
# split train, test data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

X_train.reset_index(inplace=True, drop=True)
X_test.reset_index(inplace=True, drop=True)
y_train.reset_index(inplace=True, drop=True)
y_test.reset_index(inplace=True, drop=True)

# Modeling

## Decision Tree Regressor

In [ ]:
# 너무 오래걸려서 수기로 몇개만 해봄
# DTR = DecisionTreeRegressor()

# param_grid = {"criterion": ["squared_error", "absolute_error"],
#               "min_samples_split": [10, 20, 40],
#               "max_depth": [2, 6, 8],
#               "min_samples_leaf": [20, 40, 100],
#               "max_leaf_nodes": [5, 20, 100],
#               }

# grid_cv_DTR = GridSearchCV(DTR, param_grid, cv=5)

# grid_cv_DTR.fit(X_train, y_train)

In [127]:
# 전년도 논문과 같은 평가기준으로 한번 해봄. 조금 더 좋은 결과 얻은 듯 함
# 전년도 논문에서는 input과 output이 겹쳐서 성능이 좋았던 것인듯...?
dtm = DecisionTreeRegressor(max_depth=1000000,
                           min_samples_split=2,
                           max_leaf_nodes=100000000)

dtm.fit(X_train,y_train)
print("R-Squared on train dataset={}".format(round(dtm.score(X_train,y_train), 4)))
print("R-Squared on test dataset={}".format(round(dtm.score(X_test,y_test), 4)))

R-Squared on train dataset=1.0
R-Squared on test dataset=0.7485


## Random Forest Regressor
- data가 더 많을때는 이게 더 효과적이라길래 이걸로 해봄

# Model evaluation

In [5]:
# % pip install dtreeviz

In [123]:
for i in output_features:
    print(i)

CAL_NOxFlowrateLNTOutletValid_gps
AUX_O2FracLNTOutlet_volPerc
AUX_SensTempLNTOutlet_degC
